In [11]:
from Bio import SeqIO
import os

# Define the path to our OTUs file
otus_path = os.path.join("..", "data", "processed", "otus.fasta")

# A list to hold the OTU IDs
otu_ids = []

# Loop through the fasta file and collect the IDs
for record in SeqIO.parse(otus_path, "fasta"):
    otu_ids.append(record.id)

# Print the results
print(f"Success! We have read the OTU file.")
print(f"Total number of unique OTUs (potential species) discovered: {len(otu_ids)}")
print("\nHere are the IDs of the first 10 OTUs:")
for otu_id in otu_ids[:10]:
    print(otu_id)

Success! We have read the OTU file.
Total number of unique OTUs (potential species) discovered: 31263

Here are the IDs of the first 10 OTUs:
SRR35536287.8657;size=71
SRR35536287.9396;size=45
SRR35536287.8139;size=42
SRR35536287.9307;size=39
SRR35536287.17121;size=25
SRR35536287.8054;size=17
SRR35536287.9986;size=14
SRR35536287.10544;size=13
SRR35536287.8298;size=13
SRR35536287.1129;size=12


In [12]:
import gzip
import pandas as pd
from Bio import SeqIO
import os

# --- (File finding logic remains the same) ---
pr2_path_gz = os.path.join("..", "data", "raw", "pr2_version_5.1.0_SSU_dada2.fasta.gz")
pr2_path_fasta = os.path.join("..", "data", "raw", "pr2_version_5.1.0_SSU_dada2.fasta")

if os.path.exists(pr2_path_gz):
    pr2_path = pr2_path_gz
    opener = gzip.open
elif os.path.exists(pr2_path_fasta):
    pr2_path = pr2_path_fasta
    opener = open
else:
    raise FileNotFoundError("Could not find the PR2 database file!")

print(f"Reading PR2 database from: {pr2_path}")

# --- (The new, simplified parsing logic) ---
pr2_data = []

with opener(pr2_path, "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        sequence = str(record.seq)
        
        # The entire header is the taxonomy string
        taxonomy_str = record.id
        
        # Split the taxonomy string into levels
        taxonomy_levels = taxonomy_str.split(';')
        
        # We will extract the main taxonomic ranks.
        # We check the length to avoid errors on incomplete taxonomies.
        kingdom = taxonomy_levels[0] if len(taxonomy_levels) > 0 else 'Unknown'
        phylum = taxonomy_levels[2] if len(taxonomy_levels) > 2 else 'Unknown'
        tax_class = taxonomy_levels[4] if len(taxonomy_levels) > 4 else 'Unknown'
        
        pr2_data.append([sequence, kingdom, phylum, tax_class])

# --- (DataFrame creation remains the same) ---
pr2_df = pd.DataFrame(pr2_data, columns=['sequence', 'kingdom', 'phylum', 'class'])

print("\nSuccessfully parsed the PR2 database!")
print(f"Total reference sequences loaded: {len(pr2_df)}")
print("\nHere's a preview of the first 5 entries:")
print(pr2_df.head())

print("\nLet's look at the different Kingdoms found:")
print(pr2_df['kingdom'].value_counts())

Reading PR2 database from: ..\data\raw\pr2_version_5.1.0_SSU_dada2.fasta

Successfully parsed the PR2 database!
Total reference sequences loaded: 240200

Here's a preview of the first 5 entries:
                                            sequence    kingdom        phylum  \
0  ATGCTTGTCTCAAAGATTAAGCCATGCATGTCTCAGTATAAGCTTT...  Eukaryota     Alveolata   
1  TGATCCTGCCAGTAGTCATATGCTTGTCTCAAAGATTAAGCCATGC...  Eukaryota     Alveolata   
2  AAGGGCGACGACAGATGTATCGGTGAATGAGGCTTCGGGCCTGGGG...  Eukaryota  Opisthokonta   
3  GTGCCAGCAGCCGCGGTAATTCCAGCTCCAATAGCGCATATTAAAG...  Eukaryota  Opisthokonta   
4  TGCATGTCTAAGCACATGCCTTATACGGTGAAGCCGCGAATAGCTC...  Eukaryota  Opisthokonta   

         class  
0  Dinophyceae  
1  Dinophyceae  
2   Ascomycota  
3   Ascomycota  
4     Nematoda  

Let's look at the different Kingdoms found:
kingdom
Eukaryota         223209
Bacteria            8028
Eukaryota:plas      6765
Eukaryota:mito      1890
Eukaryota:nucl       148
Archaea              131
Eukaryota:api

In [13]:
# --- 1. Filter for Eukaryotic sequences ---
# We use .str.startswith('Eukaryota') to keep all variations like Eukaryota:mito
is_eukaryota = pr2_df['kingdom'].str.startswith('Eukaryota')
euk_df = pr2_df[is_eukaryota].copy()

print(f"Original PR2 count: {len(pr2_df)}")
print(f"Filtered Eukaryotic count: {len(euk_df)}")

# --- 2. Drop rows with 'Unknown' phylum or class ---
# These are not useful for training a classifier
euk_df = euk_df[(euk_df['phylum'] != 'Unknown') & (euk_df['class'] != 'Unknown')]

print(f"Count after removing 'Unknown' labels: {len(euk_df)}")

# --- 3. Let's see the most common classes we have to work with ---
print("\nTop 20 most abundant classes in the reference database:")
print(euk_df['class'].value_counts().head(20))

Original PR2 count: 240200
Filtered Eukaryotic count: 232041
Count after removing 'Unknown' labels: 232041

Top 20 most abundant classes in the reference database:
class
Ascomycota           35203
Arthropoda           22945
Basidiomycota        11833
Syndiniales           9003
Dinophyceae           8321
Embryophyceae         7422
Coccidiomorphea       7204
Globothalamea         5723
Nematoda              5249
Gregarinomorphea      5194
Kinetoplastea         5127
Chlorophyceae         5103
Spirotrichea          4205
Mucoromycota          4130
Mollusca              3517
Polycystinea          3453
Trebouxiophyceae      3380
Opalozoa              3343
Oligohymenophorea     3242
Bacillariophyceae     3035
Name: count, dtype: int64


In [14]:
import numpy as np
from itertools import product

# --- K-mer generation function ---
def get_kmer_features(sequence, k=6):
    """
    Converts a DNA sequence into a dictionary of its k-mer counts.
    
    Args:
        sequence (str): The DNA sequence (e.g., 'AGATTAC').
        k (int): The length of the k-mer.
        
    Returns:
        dict: A dictionary where keys are k-mers and values are their counts.
    """
    kmer_counts = {}
    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i+k]
        kmer_counts[kmer] = kmer_counts.get(kmer, 0) + 1
    return kmer_counts

# --- Test the function ---
test_sequence = "AGATTACAGATTAC"
test_kmers = get_kmer_features(test_sequence, k=6)
print("--- Testing the k-mer function ---")
print(f"Sequence: {test_sequence}")
print(f"K-mer counts (k=6): {test_kmers}")

--- Testing the k-mer function ---
Sequence: AGATTACAGATTAC
K-mer counts (k=6): {'AGATTA': 2, 'GATTAC': 2, 'ATTACA': 1, 'TTACAG': 1, 'TACAGA': 1, 'ACAGAT': 1, 'CAGATT': 1}


In [15]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder

# --- 1. Select the Top N classes ---
N_CLASSES = 10
top_n_classes = euk_df['class'].value_counts().nlargest(N_CLASSES).index.tolist()
print(f"--- Using the following Top {N_CLASSES} classes for training ---\n{top_n_classes}\n")

# Filter the DataFrame to keep only these top classes
top_n_df = euk_df[euk_df['class'].isin(top_n_classes)]

# --- 2. Create a balanced subsample ---
SAMPLES_PER_CLASS = 2000 # You can adjust this number
balanced_df = top_n_df.groupby('class', group_keys=False).apply(lambda x: x.sample(SAMPLES_PER_CLASS))

print(f"--- Created a balanced dataset with {SAMPLES_PER_CLASS} samples per class ---")
print("New class distribution:")
print(balanced_df['class'].value_counts())


# --- 3. Apply the k-mer function ---
# This might take a minute or two to run
print("\n--- Applying k-mer function to all sequences... ---")
balanced_df['kmer_counts'] = balanced_df['sequence'].apply(lambda seq: get_kmer_features(seq, k=6))
print("Done applying k-mer function.")


# --- 4. Vectorize the features and labels ---
print("\n--- Vectorizing features and labels for ML ---")

# The DictVectorizer turns our dictionaries of k-mer counts into a numerical matrix
vectorizer = DictVectorizer(sparse=False)
X = vectorizer.fit_transform(balanced_df['kmer_counts'])

# The LabelEncoder turns our class names (e.g., 'Ascomycota') into numbers (e.g., 0)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(balanced_df['class'])

print(f"Our feature matrix 'X' has shape: {X.shape}")
print(f"Our label vector 'y' has shape: {y.shape}")
print(f"\nExample of a vectorized sequence (first 10 features): \n{X[0, :10]}")
print(f"\nExample of an encoded label: \n'{balanced_df['class'].iloc[0]}' is encoded as {y[0]}")

--- Using the following Top 10 classes for training ---
['Ascomycota', 'Arthropoda', 'Basidiomycota', 'Syndiniales', 'Dinophyceae', 'Embryophyceae', 'Coccidiomorphea', 'Globothalamea', 'Nematoda', 'Gregarinomorphea']



C:\Users\Kartikesh\AppData\Local\Temp\ipykernel_11552\2685057892.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = top_n_df.groupby('class', group_keys=False).apply(lambda x: x.sample(SAMPLES_PER_CLASS))


--- Created a balanced dataset with 2000 samples per class ---
New class distribution:
class
Arthropoda          2000
Ascomycota          2000
Basidiomycota       2000
Coccidiomorphea     2000
Dinophyceae         2000
Embryophyceae       2000
Globothalamea       2000
Gregarinomorphea    2000
Nematoda            2000
Syndiniales         2000
Name: count, dtype: int64

--- Applying k-mer function to all sequences... ---
Done applying k-mer function.

--- Vectorizing features and labels for ML ---
Our feature matrix 'X' has shape: (20000, 11737)
Our label vector 'y' has shape: (20000,)

Example of a vectorized sequence (first 10 features): 
[0. 0. 1. 0. 1. 0. 1. 0. 0. 0.]

Example of an encoded label: 
'Arthropoda' is encoded as 0


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# --- 1. Split the data into training and testing sets ---
# We'll use 80% for training and 20% for testing.
# random_state ensures that the split is the same every time we run the code.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"--- Data Split ---")
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

# --- 2. Initialize and Train the Model ---
# We'll use Logistic Regression, a solid and fast classifier.
# max_iter is increased to ensure the model has enough time to converge.
print("\n--- Training the Logistic Regression model... ---")
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)
print("Done training.")

# --- 3. Evaluate the Model ---
print("\n--- Evaluating model performance on the unseen test set... ---")
# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}") # .4f formats the number to 4 decimal places

# --- 4. Detailed Performance Report ---
# The classification report gives us precision, recall, and f1-score for each class.
print("\n--- Detailed Classification Report ---")
# We need to use the label_encoder to get the original class names back for the report
class_names = label_encoder.classes_
print(classification_report(y_test, y_pred, target_names=class_names))

--- Data Split ---
Training set size: 16000 samples
Testing set size: 4000 samples

--- Training the Logistic Regression model... ---
Done training.

--- Evaluating model performance on the unseen test set... ---

Model Accuracy: 0.9960

--- Detailed Classification Report ---
                  precision    recall  f1-score   support

      Arthropoda       0.99      1.00      1.00       400
      Ascomycota       0.99      1.00      0.99       400
   Basidiomycota       1.00      0.99      0.99       400
 Coccidiomorphea       1.00      0.99      1.00       400
     Dinophyceae       1.00      0.99      0.99       400
   Embryophyceae       0.99      0.99      0.99       400
   Globothalamea       1.00      1.00      1.00       400
Gregarinomorphea       1.00      1.00      1.00       400
        Nematoda       1.00      1.00      1.00       400
     Syndiniales       0.99      1.00      0.99       400

        accuracy                           1.00      4000
       macro avg       1.

In [17]:
# --- 1. Load our OTU data from otus.fasta ---
print("--- Loading our discovered OTUs from the deep-sea sample ---")
otus_path = os.path.join("..", "data", "processed", "otus.fasta")

otu_data = []
for record in SeqIO.parse(otus_path, "fasta"):
    # The header looks like 'SRR35536287.8657;size=71'
    header_parts = record.id.split(';size=')
    otu_id = header_parts[0]
    # The size is the number of reads for this OTU, its abundance
    abundance = int(header_parts[1])
    sequence = str(record.seq)
    otu_data.append([otu_id, abundance, sequence])

otu_df = pd.DataFrame(otu_data, columns=['otu_id', 'abundance', 'sequence'])
print(f"Loaded {len(otu_df)} OTUs.")

# --- 2. Apply the same k-mer function ---
print("\n--- Applying k-mer function to our OTUs... ---")
otu_df['kmer_counts'] = otu_df['sequence'].apply(lambda seq: get_kmer_features(seq, k=6))

# --- 3. Vectorize OTUs using the EXISTING trained vectorizer ---
# We use .transform() here, NOT .fit_transform(), because the vocabulary is already learned.
print("--- Vectorizing OTU features... ---")
X_otus = vectorizer.transform(otu_df['kmer_counts'])

# --- 4. Make Predictions with our trained model ---
print("--- Making predictions on the OTU data... ---")
otu_predictions_encoded = model.predict(X_otus)

# --- 5. Decode the predictions back to class names ---
otu_predictions_decoded = label_encoder.inverse_transform(otu_predictions_encoded)

# --- 6. Create the final report ---
print("--- Generating final report... ---\n")
otu_df['predicted_class'] = otu_predictions_decoded

# Sort the results by abundance to see the most common organisms first
final_report = otu_df.sort_values(by='abundance', ascending=False)

# Display the Top 20 most abundant classified OTUs
print("--- Top 20 Most Abundant OTUs and their Predicted Class ---")
print(final_report[['otu_id', 'abundance', 'predicted_class']].head(20))

# --- Bonus: Summary of biodiversity found ---
print("\n--- Summary of Predicted Classes in the Sample (by abundance) ---")
# This groups by the predicted class and sums up the abundance of all OTUs in that class
biodiversity_summary = final_report.groupby('predicted_class')['abundance'].sum().sort_values(ascending=False)
print(biodiversity_summary)

--- Loading our discovered OTUs from the deep-sea sample ---
Loaded 31263 OTUs.

--- Applying k-mer function to our OTUs... ---
--- Vectorizing OTU features... ---
--- Making predictions on the OTU data... ---
--- Generating final report... ---

--- Top 20 Most Abundant OTUs and their Predicted Class ---
               otu_id  abundance   predicted_class
0    SRR35536287.8657         71     Globothalamea
1    SRR35536287.9396         45   Coccidiomorphea
2    SRR35536287.8139         42     Globothalamea
3    SRR35536287.9307         39     Globothalamea
4   SRR35536287.17121         25   Coccidiomorphea
5    SRR35536287.8054         17     Globothalamea
6    SRR35536287.9986         14          Nematoda
7   SRR35536287.10544         13        Ascomycota
8    SRR35536287.8298         13        Ascomycota
9    SRR35536287.1129         12   Coccidiomorphea
10  SRR35536287.15462         11     Globothalamea
11  SRR35536287.15815         11        Ascomycota
12  SRR35536287.11353          

In [18]:
import joblib
import os

# --- Create a directory to save our model assets ---
model_dir = os.path.join("..", "models")
os.makedirs(model_dir, exist_ok=True) # exist_ok=True prevents an error if the folder already exists

# --- Save the four essential objects ---
joblib.dump(model, os.path.join(model_dir, 'tax_classifier.joblib'))
joblib.dump(vectorizer, os.path.join(model_dir, 'kmer_vectorizer.joblib'))
joblib.dump(label_encoder, os.path.join(model_dir, 'label_encoder.joblib'))

# We also save the test data for consistent validation
np.savez(os.path.join(model_dir, 'test_data.npz'), X_test=X_test, y_test=y_test)

print(f"Model and assets saved to the '{model_dir}' directory.")

Model and assets saved to the '..\models' directory.
